In [3]:
import unittest
import time
import pandas as pd
from unittest.mock import patch
from difflib import get_close_matches  # For partial matching
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Load the dataset
df = pd.read_csv('new_complete_responses.csv')

# Check the first few rows to ensure the data is loaded correctly
print("Dataset loaded:")
print(df.head())

# Function to interact with chatbot, including backend call handling
def interact_with_chatbot(user_input):
    if not isinstance(user_input, str) or not user_input.strip():
        return "Sorry, I don't understand that."  # Handle empty or non-string inputs
    
    user_input_lower = user_input.lower()
    
    # Handle special backend call
    if "get student record" in user_input_lower:
        return get_student_record()  # Fetch the student record from the backend
    
    # Exact match in dataset
    response_row = df[df['Question'].str.lower() == user_input_lower]
    if not response_row.empty:
        return response_row.iloc[0]['Response']
    
    # Partial match
    question_list = df['Question'].str.lower().tolist()
    closest_matches = get_close_matches(user_input_lower, question_list, n=1, cutoff=0.6)
    if closest_matches:
        closest_match = closest_matches[0]
        response_row = df[df['Question'].str.lower() == closest_match]
        return response_row.iloc[0]['Response']
    
    # Default fallback
    return "Sorry, I don't understand that."

# Dummy function to simulate backend behavior
def get_student_record():
    return {"name": "John Doe", "grade": "A"}

# Load test cases from DataFrame
def load_test_cases_from_dataframe(df):
    return [(row['Question'], row['Response']) for _, row in df.iterrows()]

class TestChatbotWithDataset(unittest.TestCase):

    @classmethod
    def setUpClass(cls):
        """Set up once for all tests."""
        sampled_test_cases = df[['Question', 'Response']].sample(n=150, random_state=42)
        cls.test_cases = load_test_cases_from_dataframe(sampled_test_cases)
        logging.info("Test setup complete. Sampled 150 test cases.")

    # Response Accuracy Test
    def test_response_accuracy_with_dataset(self):
        for user_input, expected_response in self.test_cases:
            with self.subTest(user_input=user_input):
                actual_response = interact_with_chatbot(user_input)
                self.assertEqual(actual_response, expected_response,
                                 f"Failed for input: {user_input}")

    # Performance Testing (with average response time)
    def test_performance(self):
        total_time = 0
        for user_input, _ in self.test_cases:
            start_time = time.time()
            interact_with_chatbot(user_input)
            response_time = time.time() - start_time
            total_time += response_time
            self.assertLess(response_time, 1.0, f"Response took too long for input: {user_input}")
        average_time = total_time / len(self.test_cases)
        logging.info(f"Average response time: {average_time:.4f} seconds")

    # Backend Integration Testing
    @patch(__name__ + '.get_student_record')
    def test_backend_integration(self, mock_backend_call):
        mock_backend_call.return_value = {"name": "John Doe", "grade": "A"}
        actual_response = interact_with_chatbot("Get student record")
        expected_response = {"name": "John Doe", "grade": "A"}
        self.assertEqual(actual_response, expected_response,
                         "Failed backend integration test")

    # Error Handling Test
    def test_error_handling(self):
        # Test unknown input
        unknown_input = "Blah Blah"
        expected_response = "Sorry, I don't understand that."
        actual_response = interact_with_chatbot(unknown_input)
        self.assertEqual(actual_response, expected_response, "Failed error handling test")

        # Test empty input
        empty_input = ""
        actual_response = interact_with_chatbot(empty_input)
        self.assertEqual(actual_response, expected_response, "Failed empty input handling")

        # Test non-string input
        non_string_input = 12345
        actual_response = interact_with_chatbot(non_string_input)
        self.assertEqual(actual_response, expected_response, "Failed non-string input handling")

    # Conversation Flow Test
    def test_conversation_flow(self):
        for user_input, expected_response in self.test_cases:
            with self.subTest(user_input=user_input):
                actual_response = interact_with_chatbot(user_input)
                self.assertEqual(actual_response, expected_response,
                                 f"Failed conversation flow for input: {user_input}")

    @classmethod
    def tearDownClass(cls):
        logging.info("Test suite finished.")

# Running the tests in Jupyter
def run_tests():
    suite = unittest.TestLoader().loadTestsFromTestCase(TestChatbotWithDataset)
    unittest.TextTestRunner(verbosity=2).run(suite)

run_tests()


2024-12-04 19:55:22,448 - INFO - Test setup complete. Sampled 150 test cases.
test_backend_integration (__main__.TestChatbotWithDataset.test_backend_integration) ... ok
test_conversation_flow (__main__.TestChatbotWithDataset.test_conversation_flow) ... 

Dataset loaded:
              Category       Subcategory  \
0        Accommodation    Room Transfers   
1        Academic Help  Exam Preparation   
2  Clubs and Societies  Leadership Roles   
3        Financial Aid      Scholarships   
4          Campus Life            Events   

                                 Question  \
0   Who can I contact for room transfers?   
1   What should I do if exam preparation?   
2  Can you help me with leadership roles?   
3      Can you help me with scholarships?   
4             What should I do if events?   

                                            Response        Keyword  \
0  Good question. Here's what you need to know: R...   accomodation   
1  Good question. Here's what you need to know: F...           exam   
2  I'm happy to clarify that. Leadership roles ca...        housing   
3  That's a great question. Scholarships are avai...  mental health   
4  That's a great question. Campus events are a g...        housing   

  Urgency Level  
0  

ok
test_error_handling (__main__.TestChatbotWithDataset.test_error_handling) ... ok
test_performance (__main__.TestChatbotWithDataset.test_performance) ... 2024-12-04 19:55:22,799 - INFO - Average response time: 0.0010 seconds
ok
test_response_accuracy_with_dataset (__main__.TestChatbotWithDataset.test_response_accuracy_with_dataset) ... ok
2024-12-04 19:55:23,013 - INFO - Test suite finished.

----------------------------------------------------------------------
Ran 5 tests in 0.696s

OK
